## Initial Satelitte Data retrieval.

The following dataset was gathered from NASA FIRMs website and encases all fire anomalies throughout the year of 2019. The initial EDA that follows will narrow down the scope of our search to Northern California using the proper longitute and latitude ranges comprising a square area of approximately 70,000 km^2. All anomalies contained in the final dataframe should be over land, and also with a confidence rating of over 75%. Let's razzle dazzle.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import seaborn as sns
#import geopandas as pd
#import folium

import requests
from IPython.display import Image, display

import random

import urllib.request

import warnings
warnings.filterwarnings('ignore')

#show all columns

pd.set_option('display.max_columns', None)

In [3]:
modis_df= pd.read_csv('../data/modis_2019_United_States.csv')

In [4]:
modis_df.shape

(107213, 15)

In [5]:
modis_df.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,36.1292,-114.9293,304.2,1.2,1.1,2019-01-01,950,Aqua,MODIS,59,6.03,271.1,14.1,N,0
1,30.8237,-83.6847,318.0,1.8,1.3,2019-01-01,1600,Terra,MODIS,76,6.03,293.9,30.9,D,0
2,30.8298,-83.6779,310.6,1.8,1.3,2019-01-01,1600,Terra,MODIS,54,6.03,293.9,15.7,D,0
3,26.7338,-81.0102,327.6,1.1,1.1,2019-01-01,1601,Terra,MODIS,83,6.03,297.4,26.2,D,0
4,26.7258,-81.0230,313.3,1.1,1.1,2019-01-01,1601,Terra,MODIS,67,6.03,294.4,9.2,D,0


In [6]:
modis_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107213 entries, 0 to 107212
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   latitude    107213 non-null  float64
 1   longitude   107213 non-null  float64
 2   brightness  107213 non-null  float64
 3   scan        107213 non-null  float64
 4   track       107213 non-null  float64
 5   acq_date    107213 non-null  object 
 6   acq_time    107213 non-null  int64  
 7   satellite   107213 non-null  object 
 8   instrument  107213 non-null  object 
 9   confidence  107213 non-null  int64  
 10  version     107213 non-null  float64
 11  bright_t31  107213 non-null  float64
 12  frp         107213 non-null  float64
 13  daynight    107213 non-null  object 
 14  type        107213 non-null  int64  
dtypes: float64(8), int64(3), object(4)
memory usage: 12.3+ MB


In [7]:
modis_df.isna().sum().sum()

0

In [8]:
new_df= modis_df[(modis_df['latitude'] >= 38.0881) & (modis_df['latitude'] <= 40.8336)]

In [9]:
new_df.shape

(8541, 15)

In [10]:
df_2= new_df[(new_df['longitude'] >= -123.1208) & (new_df['longitude'] <= -120.2933)]

In [11]:
df_2.shape

(2730, 15)

In [21]:
df_3 = df_2[(df_2.confidence >= 70)]

In [22]:
df_3.shape

(1537, 15)

### 

In [23]:
df_final = df_3[(df_3.satellite == 'Terra')]

In [25]:
df_final.shape ### after brief research this last step is not necessary. Terra and Aqua satellite do not measure between fires on
#land or on water, but rather refer two different satellites that occupy two different orbitals around the earth.

(755, 15)

At this point after doing my transformations i feel like i may need more data so lets import from 2017, and 2018 as well. considering we have 1537 isntances, this will not be enough images for our eventual CNN